# Parameter estimation using qualitative data

This Notebook eplains the use of qualitative data for parameter estimation, as described in [Schmiester et al. (2019)](https://www.biorxiv.org/content/10.1101/848648v1.abstract). An example model is provided in `example_qualitative`

## Import model and create pyPESTO objective

In [3]:
import pypesto
import pypesto.petab
import pypesto.optimize as optimize

import amici
import petab
import numpy as np
from pypesto.hierarchical.optimal_scaling_solver import OptimalScalingInnerSolver
import pypesto.logging
import logging

In [4]:
petab_folder = './example_qualitative/'
yaml_file = 'Raf_Mitra_NatCom2018.yaml'

petab_problem = petab.Problem.from_yaml(petab_folder + yaml_file)

importer = pypesto.petab.PetabImporter(petab_problem)

model = importer.create_model()

2020-07-01 12:16:54.154 - amici.petab_import - INFO - Importing model ...
2020-07-01 12:16:54.161 - amici.petab_import - INFO - Model name is 'Raf_Mitra_NatCom2018OptimalScaling_3CatQual'. Writing model code to '/home/leonard/Documents/projects/pypesto_qualitative/pyPESTO/doc/example/amici_models/Raf_Mitra_NatCom2018OptimalScaling_3CatQual'.
2020-07-01 12:16:54.163 - amici.petab_import - INFO - Species: 6
2020-07-01 12:16:54.164 - amici.petab_import - INFO - Global parameters: 14
2020-07-01 12:16:54.165 - amici.petab_import - INFO - Reactions: 6
2020-07-01 12:16:54.317 - amici.petab_import - INFO - Observables: 2
2020-07-01 12:16:54.318 - amici.petab_import - INFO - Sigmas: 2
2020-07-01 12:16:54.330 - amici.petab_import - DEBUG - Adding output parameters to model: OrderedDict([('noiseParameter1_Activity', None), ('noiseParameter1_Ybar', None)])
2020-07-01 12:16:54.336 - amici.petab_import - DEBUG - Condition table: (9, 3)
2020-07-01 12:16:54.340 - amici.petab_import - DEBUG - Fixed par

running build_ext
building 'Raf_Mitra_NatCom2018OptimalScaling_3CatQual._Raf_Mitra_NatCom2018OptimalScaling_3CatQual' extension
swigging swig/Raf_Mitra_NatCom2018OptimalScaling_3CatQual.i to swig/Raf_Mitra_NatCom2018OptimalScaling_3CatQual_wrap.cpp
swig -python -c++ -modern -outdir Raf_Mitra_NatCom2018OptimalScaling_3CatQual -I/home/leonard/Anaconda/envs/qualitative/lib/python3.8/site-packages/amici/swig -I/home/leonard/Anaconda/envs/qualitative/lib/python3.8/site-packages/amici/include -o swig/Raf_Mitra_NatCom2018OptimalScaling_3CatQual_wrap.cpp swig/Raf_Mitra_NatCom2018OptimalScaling_3CatQual.i
creating build
creating build/temp.linux-x86_64-3.8
creating build/temp.linux-x86_64-3.8/swig
gcc -pthread -B /home/leonard/Anaconda/envs/qualitative/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/leonard/Documents/projects/pypesto_qualitative/pyPESTO/doc/example/amici_models/Raf_Mitra_NatCom2018OptimalScaling_3CatQual -I/home/leo

In [7]:
# To allow for hierarchical optimization, set hierarchical=True, when creating the objective

objective = importer.create_objective(hierarchical=True)
problem = importer.create_problem(objective)

engine = pypesto.engine.SingleCoreEngine()

# pypesto.logging.log_to_console(logging.INFO)

optimizer = optimize.ScipyOptimizer(method='Nelder-Mead',
                                   options={'disp': True, 'maxiter': 500, 'maxfev': 500, 'fatol': 1e-10})

n_starts = 10

## Run optimization using optimal scaling approach

Different options can be used for the optimal scaling approach:
- method: `standard` / `reduced`
- reparameterized: `True` / `False`
- intervalConstraints: `max` / `max-min`
- minGap: Any float value

It is recommended to use the reduced method with reparameterization as it is the most efficient and robust choice.

When no options are provided, the default is the reduced and reparameterized formulation with max as interval constraint and `minGap=1e-10`.

### Run optimization using the reduced and reparameterized approach

In [9]:
problem.objective.calculator.inner_solver = OptimalScalingInnerSolver(options={'method': 'reduced',
                                                                               'reparameterized': True,
                                                                               'intervalConstraints': 'max',
                                                                               'minGap': 1e-10})

res_reduced_reparameterized = optimize.minimize(problem, n_starts=n_starts, optimizer=optimizer, engine=engine)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 11
         Function evaluations: 43
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 12
         Function evaluations: 47
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 12
         Function evaluations: 47
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 10
         Function evaluations: 39
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 12
         Function evaluations: 47
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 11
         Function evaluations: 43
Optimization terminated successful

### Run optimization using the reduced non-reparameterized approach

In [11]:
problem.objective.calculator.inner_solver = OptimalScalingInnerSolver(options={'method': 'reduced',
                                                                               'reparameterized': False,
                                                                               'intervalConstraints': 'max',
                                                                               'minGap': 1e-10})

res_reduced = optimize.minimize(problem, n_starts=n_starts, optimizer=optimizer, engine=engine)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 12
         Function evaluations: 47
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 12
         Function evaluations: 47
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 11
         Function evaluations: 43
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 11
         Function evaluations: 43
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 51
Optimization terminated successful

### Run optimization using the standard approach

In [ ]:
problem.objective.calculator.inner_solver = OptimalScalingInnerSolver(options={'method': 'standard',
                                                                               'reparameterized': False,
                                                                               'intervalConstraints': 'max',
                                                                               'minGap': 1e-10})

res_standard = optimize.minimize(problem, n_starts=n_starts, optimizer=optimizer, engine=engine)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 58
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 18
         Function evaluations: 51
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 58
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 56
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 18
         Function evaluations: 49


### Compare results

Reduced formulation leads to improved computation times

In [ ]:
time_standard = res_standard.optimize_result.get_for_key('time')
print(f"Mean computation time for standard approach: {np.mean(time_standard)}")

time_reduced = res_reduced.optimize_result.get_for_key('time')
print(f"Mean computation time for reduced approach: {np.mean(time_reduced)}")

time_reduced_reparameterized = res_reduced_reparameterized.optimize_result.get_for_key('time')
print(f"Mean computation time for reduced reparameterized approach: {np.mean(time_reduced_reparameterized)}")

All approaches yield the same objective function values

In [ ]:
from pypesto.visualize import waterfall

waterfall([res_standard, res_reduced, res_reduced_reparameterized])